<a href="https://colab.research.google.com/github/abhijadhav14/Deep-Learning-CSE5019/blob/main/RESNET_ETC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# One-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [2]:
def NiN():
    model = models.Sequential([
        layers.Conv2D(192, 5, padding="same", activation="relu", input_shape=(32,32,3)),
        layers.Conv2D(160, 1, activation="relu"),
        layers.Conv2D(96, 1, activation="relu"),
        layers.MaxPooling2D(2),

        layers.Conv2D(192, 5, padding="same", activation="relu"),
        layers.Conv2D(192, 1, activation="relu"),
        layers.Conv2D(10, 1, activation="relu"),

        layers.GlobalAveragePooling2D(),
        layers.Activation("softmax")
    ])
    return model

model = NiN()
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 28s 28ms/step - accuracy: 0.1749 - loss: 2.2489 - val_accuracy: 0.3309 - val_loss: 1.9982
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 18s 23ms/step - accuracy: 0.3239 - loss: 1.9872 - val_accuracy: 0.3814 - val_loss: 1.8024
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 18s 23ms/step - accuracy: 0.3909 - loss: 1.7734 - val_accuracy: 0.4131 - val_loss: 1.7053
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 18s 23ms/step - accuracy: 0.4326 - loss: 1.6668 - val_accuracy: 0.4456 - val_loss: 1.6257
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 19s 24ms/step - accuracy: 0.4681 - loss: 1.5854 - val_accuracy: 0.4560 - val_loss: 1.6071


In [3]:
def inception_block(x):
    p1 = layers.Conv2D(32, 1, activation="relu", padding="same")(x)

    p2 = layers.Conv2D(32, 1, activation="relu", padding="same")(x)
    p2 = layers.Conv2D(64, 3, activation="relu", padding="same")(p2)

    p3 = layers.Conv2D(16, 1, activation="relu", padding="same")(x)
    p3 = layers.Conv2D(32, 5, activation="relu", padding="same")(p3)

    p4 = layers.MaxPooling2D(3, strides=1, padding="same")(x)
    p4 = layers.Conv2D(32, 1, activation="relu", padding="same")(p4)

    return layers.concatenate([p1, p2, p3, p4])

inputs = layers.Input(shape=(32,32,3))
x = inception_block(inputs)
x = layers.MaxPooling2D(2)(x)
x = inception_block(x)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(10, activation="softmax")(x)

model = models.Model(inputs, outputs)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 25s 22ms/step - accuracy: 0.2404 - loss: 2.0117 - val_accuracy: 0.3836 - val_loss: 1.6188
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.4117 - loss: 1.5931 - val_accuracy: 0.4711 - val_loss: 1.4459
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.4783 - loss: 1.4344 - val_accuracy: 0.5195 - val_loss: 1.3237
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.5206 - loss: 1.3247 - val_accuracy: 0.5307 - val_loss: 1.2911
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.5539 - loss: 1.2491 - val_accuracy: 0.5405 - val_loss: 1.2745


In [4]:
def res_block(x, filters):
    skip = x
    x = layers.Conv2D(filters, 3, padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([x, skip])
    return layers.ReLU()(x)

inputs = layers.Input(shape=(32,32,3))
x = layers.Conv2D(64, 3, padding="same", activation="relu")(inputs)
x = res_block(x, 64)
x = res_block(x, 64)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(10, activation="softmax")(x)

model = models.Model(inputs, outputs)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 24s 23ms/step - accuracy: 0.3990 - loss: 1.6636 - val_accuracy: 0.5551 - val_loss: 1.2252
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - accuracy: 0.6024 - loss: 1.1299 - val_accuracy: 0.5691 - val_loss: 1.2199
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - accuracy: 0.6678 - loss: 0.9438 - val_accuracy: 0.6576 - val_loss: 0.9881
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - accuracy: 0.7031 - loss: 0.8557 - val_accuracy: 0.5663 - val_loss: 1.2500
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - accuracy: 0.7320 - loss: 0.7752 - val_accuracy: 0.6189 - val_loss: 1.1778


In [5]:
def dense_block(x, filters):
    y = layers.BatchNormalization()(x)
    y = layers.ReLU()(y)
    y = layers.Conv2D(filters, 3, padding="same")(y)
    return layers.concatenate([x, y])

inputs = layers.Input(shape=(32,32,3))
x = layers.Conv2D(32, 3, padding="same")(inputs)

for _ in range(3):
    x = dense_block(x, 32)

x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(10, activation="softmax")(x)

model = models.Model(inputs, outputs)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 18ms/step - accuracy: 0.3226 - loss: 1.8164 - val_accuracy: 0.3210 - val_loss: 1.8904
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.4896 - loss: 1.3981 - val_accuracy: 0.3645 - val_loss: 2.0298
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.5363 - loss: 1.2783 - val_accuracy: 0.3963 - val_loss: 1.8842
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.5651 - loss: 1.2121 - val_accuracy: 0.4518 - val_loss: 1.6470
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.5885 - loss: 1.1533 - val_accuracy: 0.4917 - val_loss: 1.3792
